# TripleGAN - MNIST + Keras

## 0. Necessities

### 0.0 Imports

In [19]:
# Maths
import numpy                                            as np

# Matplotlib
import matplotlib                                       as mp
import matplotlib.pyplot                                as pt

# Machine / Deep Learning
import tensorflow                                       as tf
import keras                                            as ks
from keras              import models                   as mls
from keras              import layers                   as lys
from keras.datasets     import mnist                    as mn
from keras.utils        import to_categorical           as tc

# Versions
print( f"Numpy .... : {np.__version__}" )
print( f"Matplotlib : {mp.__version__}" )
print( f"Tensorflow : {tf.__version__}" )
print( f"Keras .... : {ks.__version__}" )

ImportError: cannot import name 'Onehot' from 'keras.preprocessing' (/Library/Python/3.8/site-packages/keras/preprocessing/__init__.py)

### 0.1 Class Declaration : TripleGAN

In [ ]:
class TripleGAN( ks.Model ):
    def __init__( self, discriminator, generator, classifier, dimension ):
        super( TripleGAN, self ).__init__()
        self.discriminator = discriminator
        self.classifier = classifier
        self.generator = generator
        self.dimension = dimension

    def compile( self, dr_opt, gr_opt, cr_opt, loss_function ):
        super( TripleGAN, self ).compile()
        self.dr_opt = dr_opt
        self.gr_opt = gr_opt
        self.cr_opt = cr_opt
        self.loss_function = loss_function

    def train_step( self, real_images ):

        if isinstance( real_images, tuple ):
            real_images = real_images[0]

        # Sample random points in the latent space
        batch_size = tf.shape( real_images )[0]
        random_latent_vectors = tf.random.normal( shape = ( batch_size, self.dimension ) )

        # Decode them to fake images
        generated_images = self.generator( random_latent_vectors )

        # Combine them with real images
        combined_images = tf.concat(
            [generated_images, real_images],
            axis = 0
        )

        # Assemble labels discriminating real from fake images
        labels = tf.concat(
            [tf.ones( ( batch_size, 1 ) ), tf.zeros( ( batch_size, 1 ) )],
            axis = 0
        )
        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform( tf.shape( labels ) )

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator( combined_images )
            dr_loss = self.loss_function( labels, predictions )
        
        grads = tape.gradient( dr_loss, self.discriminator.trainable_weights )
        self.dr_opt.apply_gradients(
            zip( grads, self.discriminator.trainable_weights )
        )

        # Sample random points in the latent space
        random_latent_vectors = tf.random.normal( shape = ( batch_size, self.dimension ) )

        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros( ( batch_size, 1 ) )

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            gr_loss = self.loss_function(misleading_labels, predictions)

        grads = tape.gradient( gr_loss, self.generator.trainable_weights )
        self.gr_opt.apply_gradients( zip( grads, self.generator.trainable_weights ) )

        return {
            "dr_loss": dr_loss,
            "gr_loss": gr_loss
        }


### 0.2 Class Declaration : TripleGAN_Viewer

In [ ]:
### ERRONEOUS

class TripleGAN_Viewer( ks.callbacks.Callback ):
    def __init__( self, image_number = 100, dimension = 128 ):
        self.image_number = image_number
        self.dimension  = dimension

    def on_epoch_end( self, epoch, logs = None ):
        random_latent_vectors = tf.random.normal( shape = ( self.image_number, self.dimension ) )
        generated_images = self.model.generator( random_latent_vectors )
        generated_images *= 255
        generated_images.numpy()
        for i in range( self.image_number ):
            img = ks.preprocessing.image.array_to_img( generated_images[i] )
            img.save( "generated_img_{i}_{epoch}.png".format( i = i, epoch = epoch ) )

## 1. Set Up Model

### 1.0 Set Up Discriminator

In [22]:
# Set up discriminator



dr = mls.Sequential( name = "Discriminator" )

dr.add( lys.Input( shape = ( 10, ) ) )
dr.add( lys.Dense( 1000 ) )
dr.add( lys.LeakyReLU() )
dr.add( lys.Dense( 500 ) )
dr.add( lys.LeakyReLU() )
dr.add( lys.Dense( 250 ) )
dr.add( lys.LeakyReLU() )
dr.add( lys.Dense( 250 ) )
dr.add( lys.LeakyReLU() )
dr.add( lys.Dense( 250 ) )
dr.add( lys.LeakyReLU() )
dr.add( lys.Dense( 1, activation = "sigmoid" ) )

dr.summary()

Model: "Discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 1000)              11000     
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 1000)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 500)               500500    
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 500)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 250)               125250    
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 250)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 250)             

### 1.1 Set Up Generator

In [8]:
dimension = 128

gr = mls.Sequential( name = "Generator" )

gr.add( lys.Input( shape = ( dimension, ) ) )
gr.add( lys.Dense( 7 * 7 * 128 ) )
gr.add( lys.LeakyReLU( alpha = 0.2 ) )
gr.add( lys.Reshape( ( 7, 7, 128 ) ) )
gr.add( lys.Conv2DTranspose( 128, ( 4, 4 ), strides = ( 2, 2 ), padding = "same" ) )
gr.add( lys.LeakyReLU( alpha = 0.2 ) )
gr.add( lys.Conv2DTranspose( 128, ( 4, 4 ), strides = ( 2, 2 ), padding = "same" ) )
gr.add( lys.LeakyReLU( alpha = 0.2 ) )
gr.add( lys.Conv2DTranspose( 1, ( 7, 7 ), padding = "same", activation = "sigmoid" ) )

gr.summary()

Model: "Generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 6272)              809088    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 6272)              0         
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 28, 28, 128)       262272    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 28, 28, 128)       0 

### 1.2 Set Up Classifier

In [17]:
cr = mls.Sequential( name = "Classifier" )

cr.add( lys.Input( shape = ( dimension, 28, 28 ) ) )
cr.add( lys.Conv2D( 32, ( 5, 5 ), activation = "relu", padding = "same" ) )
cr.add( lys.MaxPooling2D( pool_size = ( 2, 2 ) ) )
cr.add( lys.Dropout( 0.5 ) )
cr.add( lys.Conv2D( 64, ( 3, 3 ), activation = "relu", padding = "same" ) )
cr.add( lys.Conv2D( 64, ( 3, 3 ), activation = "relu", padding = "same" ) )
cr.add( lys.MaxPooling2D( pool_size = ( 2, 2 ) ) )
cr.add( lys.Dropout( 0.5 ) )
cr.add( lys.Conv2D( 128, ( 3, 3 ), activation = "relu", padding = "same" ) )
cr.add( lys.Conv2D( 128, ( 3, 3 ), activation = "relu", padding = "same" ) )
cr.add( lys.GlobalMaxPooling2D() )
cr.add( lys.Dense( 10, activation = "softmax" ) )

cr.summary()

Model: "Classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_23 (Conv2D)           (None, 128, 28, 32)       22432     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 64, 14, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 64, 14, 32)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 64, 14, 64)        18496     
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 64, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 32, 7, 64)         0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 32, 7, 64)         0

## 2. Setup MNIST Dataset 

In [32]:
( tr_images, tr_labels ), ( ts_images, ts_labels ) = mn.load_data()

batch_size = 64

all_digits = np.concatenate( [tr_images, ts_images] )
all_digits = all_digits.astype( 'float32' ) / 255
all_digits = np.reshape( all_digits, ( -1, 28, 28, 1 ) )

dataset = tf.data.Dataset.from_tensor_slices( all_digits )
dataset = dataset.shuffle( buffer_size=1024 ).batch( batch_size ).prefetch( 32 )

## 3. Create GAN Model

In [33]:
epochs = 30

gan = MonoGAN(
    discriminator   = dr,
    generator       = gr,
    dimension       = dimension
)

## 4.  Compile GAN Model

In [34]:
gan.compile(
    dr_opt          = ks.optimizers.Adam( learning_rate = 0.0003 ),
    gr_opt          = ks.optimizers.Adam( learning_rate = 0.0003 ),
    loss_function   = ks.losses.BinaryCrossentropy( from_logits = True )
)


## 4. Train Model

In [20]:
gan.fit(
    dataset,
    epochs = epochs,
    callbacks = [ MonoGAN_Monitor( image_number = 3, dimension = dimension ) ]
)

Epoch 1/5
938/938 [==============================] - 66s 69ms/step - loss: 0.8713 - accuracy: 0.8633
Epoch 2/5
938/938 [==============================] - 60s 64ms/step - loss: 0.0243 - accuracy: 0.9948
Epoch 3/5
938/938 [==============================] - 41s 43ms/step - loss: 0.0150 - accuracy: 0.9967
Epoch 4/5
938/938 [==============================] - 46s 49ms/step - loss: 0.0168 - accuracy: 0.9961
Epoch 5/5
938/938 [==============================] - 44s 47ms/step - loss: 0.0163 - accuracy: 0.9964


## 5. Evaluate Model

In [22]:
for i in range( 3 ):
    pt.figure( figsize = ( 30, 30 ) )
    im = pt.imread( f"./generated_img_{i}_29.png" )
    pt.imshow( im )

313/313 [==============================] - 2s 8ms/step - loss: 0.0508 - accuracy: 0.9910
0.9909999966621399
